In [ ]:
%pip install openai
%pip install azure-search-documents

In [12]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient

In [ ]:
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = "index1"

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def upload_document(document):
    result = search_client.upload_documents(documents=[document])
    print("Upload of new document succeeded: {}".format(result[0].succeeded))


def merge_document():
    result = search_client.merge_documents(documents=[{"hotelId": "1000", "rating": 4.5}])
    print("Merge into new document succeeded: {}".format(result[0].succeeded))


def delete_document():
    result = search_client.delete_documents(documents=[{"hotelId": "1000"}])
    print("Delete new document succeeded: {}".format(result[0].succeeded))

def get_document(id):
    result = search_client.get_document(id)

    print("Details:")
    print("Filepath: {}".format(result["filepath"]))
    print("Title: {}".format(result["title"]))
    print("URL: {}".format(result["url"]))
    print("Content: {}".format(result["content"]))
    print("ContentVector: {}".format(result["contentVector"]))

document = {
    "content": "Blazing fast search for your Apps and Websites",
    "filepath": "/path/to/file",
    "title": "A good title",
    "url": "https://www.example.com",
    "id": "",
    "chunk_id": "",
    "last_updated": "2021-01-01T00:00:00Z",
    "contentVector": ""
}

id = "aHR0cHM6Ly9vaWEuYmxvYi5jb3JlLndpbmRvd3MubmV0L2luZGV4MS1jaHVua3MvYUhSMGNITTZMeTl2YVdFdVlteHZZaTVqYjNKbExuZHBibVJ2ZDNNdWJtVjBMMlpwYkdWMWNHeHZZV1F0YVc1a1pYZ3hMMFZ1ZEhKbGRtbHpkR0ZmVDNwcGNtVnpKVEl3TVNVeU1HUmxKVEl3TWk1d1pHWTEvY29udGVudF9jaHVua3NfMTEuanNvbg2"
get_document(id)